# Buisness Problem


# Data Understanding

# Data Preperation

In [ ]:
import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt

import gzip
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

from nltk.sentiment import SentimentIntensityAnalyzer

import re

import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

In [ ]:
import seaborn as sns

In [ ]:
import contractions

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Subsetting data

## Initial Pre-processing

# Data Exploration

## EDA

In [ ]:
df = pd.read_csv('data/reviews_subset.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.isna().sum()

In [ ]:
df.overall.value_counts(normalize = True)

In [ ]:
ax = df.overall.value_counts().sort_index().plot(kind = 'bar',
                                            title = 'Count by Review Star',
                                            figsize = (10,5))
ax.set_xlabel('Review Stars');


In [ ]:
df.verified.value_counts()

In [ ]:
sns.barplot(data = df, x = 'verified', y = 'overall')

In [ ]:
df.brand.value_counts()

In [ ]:
dfg = df.groupby('brand')['overall'].mean().sort_values(ascending = False)

In [ ]:
fig, ax = plt.subplots(figsize = (10,30))
ax = sns.barplot(x = dfg.values, y = dfg.index, orient = 'h')

## Sentiment Validation of Star Ratings

In [ ]:
sia = SentimentIntensityAnalyzer()

In [ ]:
df.reviewText[0]

In [ ]:
df.overall[0]

In [ ]:
sia.polarity_scores(df.reviewText[0])

In [ ]:
sent_val_df = df.copy()

In [ ]:
sent_val_df['compound'] = [sia.polarity_scores(str(x))['compound'] for x in df['reviewText']]
sent_val_df['neg'] = [sia.polarity_scores(str(x))['neg'] for x in df['reviewText']]
sent_val_df['neu'] = [sia.polarity_scores(str(x))['neu'] for x in df['reviewText']]
sent_val_df['pos'] = [sia.polarity_scores(str(x))['pos'] for x in df['reviewText']]

In [ ]:
sent_val_df.head()

In [ ]:
ax = sns.barplot(data = sent_val_df, x = 'overall', y = 'compound')

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15,5))
sns.barplot(data = sent_val_df, x = 'overall', y = 'pos', ax = axs[0])
sns.barplot(data = sent_val_df, x = 'overall', y = 'neu', ax = axs[1])
sns.barplot(data = sent_val_df, x = 'overall', y = 'neg', ax = axs[2])
axs[0].set_title('Positive')
axs[1].set_title('Neutral')
axs[2].set_title('Negative');

In [ ]:
def label_encode(x):
    if x <= 3:
        return 0
    if x > 3:
        return 1

In [ ]:
df["sentiment"] = df["overall"].apply(lambda x: label_encode(x))

In [ ]:
df['sentiment'].value_counts(normalize = True)

## Text Exploration

In [ ]:
def word_count(text):
    return len(word_tokenize(text))

def char_count(text):
    text_without_spaces = text.replace(" ", "")
    return len(text_without_spaces)

In [ ]:
df['word_count'] = df.reviewText.apply(lambda x: word_count(x))
df['review_len'] = df.reviewText.apply(lambda x: char_count(x))

In [ ]:
df.head()

In [ ]:
text_stats_df = df[['reviewText', 'sentiment', 'word_count', 'review_len']].copy()

In [ ]:
text_stats_df.head()

In [ ]:
text_stats_df.groupby('sentiment')['word_count'].mean().plot(kind = 'bar', figsize = (15,15));

In [ ]:
text_stats_df.groupby('sentiment')['review_len'].mean().plot(kind = 'bar', figsize = (15,15));

In [ ]:
correlation = text_stats_df[['sentiment', 'word_count', 'review_len']].corr()
mask = np.zeros_like(correlation, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(50,30))
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
sns.heatmap(correlation, cmap='coolwarm', annot=True, annot_kws={"size": 40}, linewidths=10, vmin=-1.5, mask=mask);

In [ ]:
text_stats_df.describe()

In [ ]:
text_stats_df.word_count.hist();

In [ ]:
text_stats_df.review_len.hist();

In [ ]:
text_stats_df.sort_values(by = 'word_count', ascending = False).head(20)

In [ ]:
text_stats_df.sort_values(by = 'word_count', ascending = False).tail(20)

In [ ]:
text_stats_df.reviewText[44098]

In [ ]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Fix contractions
    text = contractions.fix(text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove HTML tags
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    # Remove line breaks and extra whitespace
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
    #tokenize and lemmatize
    tokens = [token.lemma_ for token in nlp(text)]
    # Strip leading/trailing whitespace and join tokens
    clean_text = ' '.join(tokens).strip()
    return clean_text

In [ ]:
text_stats_df.reviewText[44098]

In [ ]:
clean_text(text_stats_df.reviewText[44098])

In [ ]:
text_stats_df['cleaned_text'] = text_stats_df.reviewText.apply(lambda x: clean_text(x))

In [ ]:
text_stats_df.head()

In [ ]:
text_stats_df.cleaned_text[44098]

In [ ]:
text_stats_df['cleaned_word_count'] = text_stats_df.cleaned_text.apply(lambda x: word_count(x))
text_stats_df['cleaned_review_len'] = text_stats_df.cleaned_text.apply(lambda x: char_count(x))

In [ ]:
text_stats_df.describe()

In [ ]:
text_stats_df[text_stats_df.cleaned_word_count == 0]

In [ ]:
data_df = text_stats_df[text_stats_df.cleaned_word_count >0].copy()

In [ ]:
data_df.describe()

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
def count_stop_words(text):
    # Tokenize the text into words
    words = word_tokenize(text)

    # Convert the stop words set to lower case
    stop_words_lower = set([word.lower() for word in stop_words])

    # Count the number of stop words in the text using set intersection
    stop_word_count = len(set(words).intersection(stop_words_lower))

    return stop_word_count

In [ ]:
data_df['cleaned_sw_count'] = data_df.cleaned_text.apply(lambda x: count_stop_words(x))

In [ ]:
data_df.describe()

In [ ]:
cv = CountVectorizer(min_df = 10, ngram_range = (3,3), stop_words = stop_words)
cv.fit_transform(data_df.cleaned_text)


In [ ]:
def top_words(text, n = None, ngram = (1,1), stop_words = None):
    cv = CountVectorizer(ngram_range = ngram, stop_words = stop_words)
    words = cv.fit_transform(text)
    sum_words = words.sum(axis=0)
    word_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
    word_freq = sorted(word_freq, key=lambda x: x[1], reverse=True)
    word_df = pd.DataFrame(word_freq, columns=['word', 'count'])
    return word_df[:n]

In [ ]:
top30 = top_words(text = data_df.cleaned_text, n = 30)
top30_sw = top_words(text = data_df.cleaned_text, n = 30, stop_words = stop_words)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,10))
sns.barplot(data = top30, x = 'count', y = 'word', orient = 'h', ax = axs[0])
sns.barplot(data = top30_sw, x = 'count', y = 'word', orient = 'h', ax = axs[1])
axs[0].set_title('All words')
axs[1].set_title('Stop Words Removed');

In [ ]:
top30_bigrams = top_words(text = data_df.cleaned_text, n = 30, ngram = (2,2))
top30_bigrams_sw = top_words(text = data_df.cleaned_text, n = 30, ngram = (2,2), stop_words = stop_words)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,10))
sns.barplot(data = top30_bigrams, x = 'count', y = 'word', orient = 'h', ax = axs[0])
sns.barplot(data = top30_bigrams_sw, x = 'count', y = 'word', orient = 'h', ax = axs[1])
axs[0].set_title('All words Bigrams')
axs[1].set_title('Stop Words Removed Bigrams')
fig.tight_layout();

In [ ]:
top30_trigrams = top_words(text = data_df.cleaned_text, n = 30, ngram = (3,3))
top30_trigrams_sw = top_words(text = data_df.cleaned_text, n = 30, ngram = (3,3), stop_words = stop_words)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15,10))
sns.barplot(data = top30_trigrams, x = 'count', y = 'word', orient = 'h', ax = axs[0])
sns.barplot(data = top30_trigrams_sw, x = 'count', y = 'word', orient = 'h', ax = axs[1])
axs[0].set_title('All words Trigrams')
axs[1].set_title('Stop Words Removed Trigrams')
fig.tight_layout();

In [ ]:
data_df.describe()

In [ ]:
data_df.cleaned_word_count.hist()

In [ ]:
data_df[data_df.cleaned_word_count <= 1000]

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
def stop_token(text):
    tokens = word_tokenize(text)
    filtered_token = [token for token in tokens if token not in stop_words]
    return filtered_token

In [ ]:
data_words = data_df.cleaned_text.apply(lambda x: stop_token(x))

In [ ]:
data_words

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { max-width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.output_area { max-width:100% !important; }</style>"))
display(HTML("<style>.input_area { max-width:100% !important; }</style>"))

In [ ]:
from gensim.models.phrases import Phrases, Phraser

# Create bigrams
bigram = Phrases(data_words, min_count=5, threshold=50)
bigram_phraser = Phraser(bigram)
bigram_docs = [bigram_phraser[doc] for doc in data_words]

# Create trigrams
trigram = Phrases(bigram_docs, min_count=5, threshold=50)
trigram_phraser = Phraser(trigram)
trigram_docs = [trigram_phraser[doc] for doc in bigram_docs]


In [ ]:
id2word = corpora.Dictionary(trigram_docs)

# Create Corpus
texts = trigram_docs

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=5,
                                           alpha='symmetric',
                                           per_word_topics=True)

In [ ]:
cv = CountVectorizer(min_df = 10, ngram_range = (1,3), stop_words = stop_words)
X = cv.fit_transform(data_df.cleaned_text)

In [ ]:
cv.get_feature_names()

In [ ]:
id2word = corpora.Dictionary([cv.get_feature_names()])

# Convert the count matrix to gensim corpus
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Define your LDA model
num_topics = 10
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, passes=5)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, id2word)
vis